# Introduction to Cardano Data Analysis Notebook pt.1

In this notebook, we will explore the Cardano blockchain but from the perspective of an SPO with the hope of bettering the community's understanding of the centralization of the blockchain and how we can solve this problem. This notebook will be part of a series of notebooks that will be used to better understand the Cardano blockchain.

What will we cover in this series of notebooks?

- Centralization of the blockchain's stake to small number of pools
- Centralization of the blockchain's block production to small number of pools
- Centralization of the blockchain's transaction production to small number of pools
- Centralization of the blockchain's delegators to small number of pools
- Centralization of the blockchain based on ISP's used
- Number of Stake Pools at or near saturation
- Centralization based on multi-pool operation
- Centralization based on geographical location

In [33]:
import json
import time
import requests
import math as m
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import descartes
import geopandas as gpd
from shapely.geometry import Point, Polygon
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from scipy.optimize import curve_fit
import plotly.express as px
import chart_studio
import chart_studio.plotly as py
import chart_studio.tools as tls
import folium
import ast


# Leave This code here and lookup at pandas documentation 
# if you need to know about chained assignments
pd.options.mode.chained_assignment = None  # default='warn'

# Because most of my data sets I have made can have up to 30+ columns and 20+ rows
# code below will increase pandas defaults for max rows 
# and columns that you can display in a Juptyer Notebook
pd.options.display.max_columns = 60
pd.options.display.max_rows = 80


In [34]:
data = pd.read_json('relay_data_w_meta_clean.json')
data['live_stake_ada'] = data['live_stake']/1000000
data['live_pledge_ada'] = data['live_pledge']/1000000
data['live_declared_pledge_ada'] = data['declared_pledge']/1000000

# create df for ada staked per pool
df_ada_staked_per_pool = data.copy()
df_ada_staked_per_pool = df_ada_staked_per_pool.drop_duplicates(subset=['pool_id'], keep='first')

# Get the number of pools per isp
pools_per_isp = df_ada_staked_per_pool.groupby('isp').pool_id.nunique().to_frame()
pools_per_isp.rename(columns={'pool_id': 'Number of Pools'}, inplace=True)

# Get the number of ada staked per isp
ada_per_isp = (df_ada_staked_per_pool.groupby('isp').live_stake.sum()/1000000).to_frame()
ada_per_isp = ada_per_isp.merge(pools_per_isp, left_index=True, right_index=True)

# get the amount of ada staked per isp with more than 100 mil ada staked
# ada_per_isp_100mil = ada_per_isp[ada_per_isp['live_stake'] > 100000000]
ada_per_isp_100mil = ada_per_isp.loc[ada_per_isp['live_stake'] > 100000000]
ada_per_isp_100mil.sort_values(by='live_stake', ascending=False, inplace=True)
ada_per_isp_100mil =  ada_per_isp_100mil.reset_index().replace({'isp': {'amazon.com Inc.':'amazon.com, Inc'}}).groupby('isp', sort=False).sum()
ada_per_isp_100mil.rename(index={'': 'Unknown'}, inplace=True)

# Get the top 100 isps by number of pools and stake
num_relays_per_isp = data.groupby(['isp']).size().to_frame()
num_relays_per_isp.rename(columns={0:'Number of Relays'}, inplace=True)
top_100_isp = num_relays_per_isp.head(100)
top_100_isp = top_100_isp.reset_index().replace({'isp': {'amazon.com Inc.':'amazon.com, Inc'}}).groupby('isp', sort=False).sum()
top_100_isp.sort_values(by='Number of Relays', ascending=False, inplace=True)



In [35]:
ada_per_pool = df_ada_staked_per_pool.groupby('pool_id').live_stake.sum().to_frame()
ada_per_pool.rename(columns={'live_stake': 'ADA Staked'}, inplace=True)

## Let's get started!

First thing we are going to do is basically explore various data points on the cardano blockchain to get a feel for the overall block production, stake, tx fees, delegators, pledge, and other data points. We will explore simple statistics to help us describe the central tendency of the data, but we need to first describe some important outlines to use in describing the centralization vs decentralization of Cardano. 

If we start with IOHK's D parameter, we can see that the D parameter is a measure of the number of blocks that are produced by federated IOHK nodes vs blocks produced by independent publicly owned stake pool nodes. The formula is D = (N_blocks_per_federated_node_per_epoch) / (N_blocks_per_stake_pool_node_per_epoch). Currently D = 0, which according to this simple metric would indicate that the cardano blockchain is fully decentralized, but only when it comes to block production. Most of us in the cardano stake pool operator community are well aware this is not the case when looking at many other data metrics on cardano blockchain for example distribution of block production, stake, delegators, and more would indicate otherwise. You will see that a small percentage of pools actually control the blockchains production/security at this moment in time. 

In [36]:
df2 = df_ada_staked_per_pool[['live_stake_ada', 'live_pledge_ada', 'blocks_minted', 'live_delegators']]
df2

,live_stake_ada,live_pledge_ada,blocks_minted,live_delegators
0,3.212853e+07,574564.954842,4097,3170
1,1.942799e+07,567027.531192,1824,170
2,5.672152e+05,546041.471277,80,17
3,1.735455e+06,207011.830413,259,724
5,4.000925e+03,0.000000,23,14
...,...,...,...,...
5545,9.957634e+04,357.225752,0,2
5547,2.008504e+02,200.850366,0,1
5549,1.500601e+03,1500.600798,0,1
5550,2.036975e+00,2.036975,0,1


In [37]:
stats = df2.describe()
stats

,live_stake_ada,live_pledge_ada,blocks_minted,live_delegators
count,3.039000e+03,3.039000e+03,3039.000000,3039.000000
mean,7.359105e+06,3.658211e+05,547.306680,340.084238
std,1.659242e+07,3.815235e+06,1188.224215,1774.924625
min,0.000000e+00,0.000000e+00,0.000000,0.000000
25%,1.283558e+03,4.538771e+02,0.000000,2.000000
50%,8.250328e+04,8.247425e+03,7.000000,7.000000
75%,3.129024e+06,6.336995e+04,307.000000,47.500000
max,8.585759e+07,6.663534e+07,9441.000000,32235.000000


- **In the above output we describe the data via the use of statistical metrics namely the count(number of pools in dataset), mean, standard deviation, range, and percentiles. What is unsettling is seeing the standard deviations being so large for each category (column)**

- **We can see that the standard deviation for the count is very high, meaning that there are a lot of outliers in the data. This is a sign that the data is not representative of the real world.**

- **When looking specifically at the percentiles on the delegator column we see another troubling trend, we see that 75% of the pools have 48 delegators or less.**

Let's take a look at the number of pools in the data set that are above the threshold of 48 delegators.

In [38]:
pools_w_more_48_delegators = df_ada_staked_per_pool.loc[df_ada_staked_per_pool['live_delegators'] > 48]
pools_w_more_48_delegators.live_stake_ada.sum()

14693069522.543468

In [39]:
p = pd.Series(pools_w_more_48_delegators.pool_id)

In [40]:
p.to_csv('pools_w_more_48_delegators.csv', index=False)

# Let's make up some decentralization metrics

- First thing I want to do is make a metric for isp's that are used in the data set. For example the large % of pools that use the ISP 'Telecom Italia' the worse this metric would affect the overall score.

- Next, I want to make a metric for pledge ratio (live_stake/declared_pledge) this will give us a "skin in the game" metric for the data set. Pools operators that often run many pools (1pct, BNB, etc) will have a higher pledge ratio because they spread the little ada they have to open more pools instead of allowing single pool operators to get some of their delegators or stake.

- The next metric will be potentially reward accounts or need some way to identify this pool runs many pools on the network under same ticker or owner. hard part is smart pools like 1pct never use the same reward account, stake address, or payment address for all of their pools so only way you can tell they run all the pools is because they say so on the website, ticker, url, or somewhere else in the onchain metadata.

In [41]:
dfx = data.groupby('isp').pool_id.nunique().to_frame().sort_values(by='pool_id', ascending=False)
dfx = dfx.reset_index().replace({'isp': {'amazon.com Inc.':'amazon.com, Inc'}}).groupby('isp', sort=False).sum()
dfx['isp_ratio'] = dfx['pool_id']/dfx.pool_id.sum() * 100
dfx

,pool_id,isp_ratio
isp,,
"amazon.com, Inc",594,16.150082
Digitalocean LLC,422,11.473627
Hetzner Online Gmbh,292,7.939097
Contabo Gmbh,266,7.232191
Google LLC,217,5.899946
...,...,...
Leaseweb Usa Inc.,1,0.027189
Leaseweb Asia Pacific Pte. Ltd.,1,0.027189
Launtel,1,0.027189


In [42]:
df_ada_staked_per_pool.groupby('reward_account').pool_id.count().to_frame().sort_values(by='pool_id', ascending=False)

,pool_id
reward_account,
stake1uy6yzwsxxc28lfms0qmpxvyz9a7y770rtcqx9y96m42cttqwvp4m5,33
stake1uxcmc9r2t7cxs0zw8qm8ztg3twvxryzghsc8eszek6kuwms23gya4,13
stake1uygvh4uq45uuavd7hfnmfvvg73ftddcglysquyre9yzsfvcaprtu5,8
stake1u9hwxtcy6xwxqf54hxfgseq54u70p492jurq7dwdxz5u8jcggs94w,5
stake1uxuewv5gf4uu28cpkg6vcg959uz7ra260e9u8465qq4358sruvdch,5
...,...
stake1u9d66r3lszhsu2x9jjjkyd4g4gsc75zlwdq4nmtj3ts22kcjp9g4p,1
stake1u9d7kktn2tak37w2dk03m3409ne8qqg0j487dv7ewp68xhgy8h5gu,1
stake1u9da2qzucsmmt93zhgrr6ygjjm2x3huedmtxxvgl77c5yrgjg00vz,1


In [43]:
data

,pool_id,hex,vrf_key,blocks_minted,live_stake,live_size,live_saturation,live_delegators,active_stake,active_size,declared_pledge,live_pledge,margin_cost,fixed_cost,reward_account,owners,registration,retirement,ip_host,port,url,hash,ticker,name,description,homepage,resolved_ip,longitude,latitude,zip,region_name,region_code,connection,city,country_name,country_code,error,success,detail,isp,asn,live_stake_ada,live_pledge_ada,live_declared_pledge_ada
0,pool1z5uqdk7dzdxaae5633fqfcu2eqzy3a3rgtuvy087f...,153806dbcd134ddee69a8c5204e38ac80448f62342f8c2...,0220a5d08adbfe9554b52d7b2993be5892ac3ff340e674...,4097,32128532235090,1.359849e-03,4.795368e-01,3170,32462689016256,0.001368,100000000000,574564954842,0.009,340000000,stake1uy89kzrdlpaz5rzu8x95r4qnlpqhd3f8mf09edjp...,['stake1uy89kzrdlpaz5rzu8x95r4qnlpqhd3f8mf09ed...,['841cca81da918feb9fa7257a34630eac95794be712ed...,[],54.220.20.40,3002,https://raw.githubusercontent.com/Octalus/card...,ca7d12decf886e31f5226b5946c62edc81a7e40af95ce7...,OCTAS,OctasPool,Octa's Performance Pool,https://octaluso.dyndns.org,54.220.20.40,-6.243330,53.353889,None,Leinster,L,"{'asn': 16509, 'isp': 'amazon.com, Inc'}",Dublin,Ireland,IE,NaN,NaN,NaN,"amazon.com, Inc",16509,3.212853e+07,574564.954842,100000.0
1,pool1pu5jlj4q9w9jlxeu370a3c9myx47md5j5m2str0na...,0f292fcaa02b8b2f9b3c8f9fd8e0bb21abedb692a6d505...,b512cc7c1a8ba689c2d8fd27adfdbac2049a3f8f95c8b8...,1824,19427988062303,8.222951e-04,2.899739e-01,170,19414895397699,0.000818,510000000000,567027531192,0.049,340000000,stake1u98nnlkvkk23vtvf9273uq7cph5ww6u2yq2389ps...,['stake1u98nnlkvkk23vtvf9273uq7cph5ww6u2yq2389...,['a96c79773b7506211eb56bf94886a2face17657d1009...,[],relays.mainnet.stakenuts.com,3001,https://stakenuts.com/mainnet.json,47c0c68cb57f4a5b4a87bad896fc274678e7aea98e200f...,NUTS,StakeNuts,StakeNuts.com,https://stakenuts.com/,198.13.62.211,139.746155,35.583511,143-0006,Kanagawa,14,"{'asn': 20473, 'isp': 'The Constant Company LLC'}",Kawasaki,Japan,JP,NaN,NaN,NaN,The Constant Company LLC,20473,1.942799e+07,567027.531192,510000.0
2,pool1c8k78ny3xvsfgenhf4yzvpzwgzxmz0t0um0h2xnn2...,c1ede3cc9133209466774d4826044e408db13d6fe6df75...,b8ba5f7dcbf17e95781e325114da46def1fa908435a27d...,80,567215180144,2.400750e-05,8.466013e-03,17,555030759623,0.000023,50100000000,546041471277,0.000,340000000,stake1uyx0ut5wc5nmu0tacjc9pyyahdyq5ujghwljkdxr...,['stake1uyx0ut5wc5nmu0tacjc9pyyahdyq5ujghwljkd...,['37f5682e2622c821175bab9d5637b7321ecacc5ec663...,[],relay.zenithpool.io,31400,https://metadata.zenithpool.io,21cc44498ab6e74e081a805e629fff171a66d0c8d034f9...,ZEN,ZENITH,"One owner, one operator, one pool. Operating s...",https://zenithpool.io,193.122.12.254,-122.333519,47.611012,98101,Washington,WA,"{'asn': 31898, 'isp': 'Oracle Corporation'}",Seattle,United States,US,NaN,NaN,NaN,Oracle Corporation,31898,5.672152e+05,546041.471277,50100.0
3,pool1q80jjs53w0fx836n8g38gtdwr8ck5zre3da90peux...,01df29429173d263c7533a22742dae19f16a08798b7a57...,b75115a1e4e32a33985fc720f3715f0e99617b448fbc36...,259,1735454735054,7.345360e-05,2.590266e-02,724,1735713797295,0.000073,200000000000,207011830413,0.005,340000000,stake1uxxsa2rlkv728pve3ks7kn7c6v68luagdjgwa0me...,['stake1uxxsa2rlkv728pve3ks7kn7c6v68luagdjgwa0...,['e8951eddb19b697bc74549aa9d4f956a8be8d5cf2c7a...,[],95.183.53.20,6060,https://ispool.live/metadata,34fdde237812fab14d29a80423bb295f39122f4fea1aae...,000,Switzerland Investment,"Stability, Security, Reliability, Neutrality! ...",https://ispool.live/,95.183.53.20,8.536950,47.374168,8043,Zurich,ZH,"{'asn': 197988, 'isp': 'Solar Communications G...",Zürich,Switzerland,CH,NaN,NaN,NaN,Solar Communications Gmbh,197988,1.735455e+06,207011.830413,200000.0
4,pool1q80jjs53w0fx836n8g38gtdwr8ck5zre3da90peux...,01df29429173d263c7533a22742dae19f16a08798b7a57...,b75115a1e4e32a33985fc720f3715f0e99617b448fbc36...,259,1735454735054,7.345360e-05,2.590266e-02,724,1735713797295,0.000073,200000000000,207011830413,0.005,340000000,stake1uxxsa2rlkv728pve3ks7kn7c6v68luagdjgwa0me...,['stake1uxxsa2rlkv728pve3

In [44]:
first_100_pools = data.head(100)
first_100_pools.groupby('homepage').pool_id.count().to_frame().sort_values(by='pool_id', ascending=False)

,pool_id
homepage,
https://www.ankachain.com,5
https://307pool.com,4
https://cardanocanucks.com,4
https://elitestakepool.com,4
https://leaf-pool.com/,4
https://cardano.ich-war-hier.de,3
https://cardanoproxy.com,3
https://staker.space,3
https://stakepool.at,3


In [45]:
dfy = data.groupby('homepage').pool_id.nunique().to_frame().sort_values(by='pool_id', ascending=False)
dfy['multipool_ratio'] = dfy['pool_id']/dfy.pool_id.sum() * 100
dfy

,pool_id,multipool_ratio
homepage,,
https://www.binance.com,62,2.448657
https://iohk.io,34,1.342812
https://www.1percentpool.eu,28,1.105845
https://memepools.io/,27,1.066351
https://emurgo.io/,17,0.671406
...,...,...
https://dodopool.io/,1,0.039494
https://docstakepool.carrd.co,1,0.039494
https://dochdoch.gitlab.io/add/,1,0.039494


In [51]:
df_ada_staked_per_pool['pledge_ratio'] = df_ada_staked_per_pool.live_stake_ada/df_ada_staked_per_pool.live_declared_pledge_ada
df_ada_staked_per_pool

,pool_id,hex,vrf_key,blocks_minted,live_stake,live_size,live_saturation,live_delegators,active_stake,active_size,declared_pledge,live_pledge,margin_cost,fixed_cost,reward_account,owners,registration,retirement,ip_host,port,url,hash,ticker,name,description,homepage,resolved_ip,longitude,latitude,zip,region_name,region_code,connection,city,country_name,country_code,error,success,detail,isp,asn,live_stake_ada,live_pledge_ada,live_declared_pledge_ada,pledge_ratio
0,pool1z5uqdk7dzdxaae5633fqfcu2eqzy3a3rgtuvy087f...,153806dbcd134ddee69a8c5204e38ac80448f62342f8c2...,0220a5d08adbfe9554b52d7b2993be5892ac3ff340e674...,4097,32128532235090,1.359849e-03,4.795368e-01,3170,32462689016256,1.368182e-03,100000000000,574564954842,0.009,340000000,stake1uy89kzrdlpaz5rzu8x95r4qnlpqhd3f8mf09edjp...,['stake1uy89kzrdlpaz5rzu8x95r4qnlpqhd3f8mf09ed...,['841cca81da918feb9fa7257a34630eac95794be712ed...,[],54.220.20.40,3002,https://raw.githubusercontent.com/Octalus/card...,ca7d12decf886e31f5226b5946c62edc81a7e40af95ce7...,OCTAS,OctasPool,Octa's Performance Pool,https://octaluso.dyndns.org,54.220.20.40,-6.243330,53.353889,None,Leinster,L,"{'asn': 16509, 'isp': 'amazon.com, Inc'}",Dublin,Ireland,IE,NaN,NaN,NaN,"amazon.com, Inc",16509,3.212853e+07,574564.954842,100000.0,321.285322
1,pool1pu5jlj4q9w9jlxeu370a3c9myx47md5j5m2str0na...,0f292fcaa02b8b2f9b3c8f9fd8e0bb21abedb692a6d505...,b512cc7c1a8ba689c2d8fd27adfdbac2049a3f8f95c8b8...,1824,19427988062303,8.222951e-04,2.899739e-01,170,19414895397699,8.182658e-04,510000000000,567027531192,0.049,340000000,stake1u98nnlkvkk23vtvf9273uq7cph5ww6u2yq2389ps...,['stake1u98nnlkvkk23vtvf9273uq7cph5ww6u2yq2389...,['a96c79773b7506211eb56bf94886a2face17657d1009...,[],relays.mainnet.stakenuts.com,3001,https://stakenuts.com/mainnet.json,47c0c68cb57f4a5b4a87bad896fc274678e7aea98e200f...,NUTS,StakeNuts,StakeNuts.com,https://stakenuts.com/,198.13.62.211,139.746155,35.583511,143-0006,Kanagawa,14,"{'asn': 20473, 'isp': 'The Constant Company LLC'}",Kawasaki,Japan,JP,NaN,NaN,NaN,The Constant Company LLC,20473,1.942799e+07,567027.531192,510000.0,38.094094
2,pool1c8k78ny3xvsfgenhf4yzvpzwgzxmz0t0um0h2xnn2...,c1ede3cc9133209466774d4826044e408db13d6fe6df75...,b8ba5f7dcbf17e95781e325114da46def1fa908435a27d...,80,567215180144,2.400750e-05,8.466013e-03,17,555030759623,2.339250e-05,50100000000,546041471277,0.000,340000000,stake1uyx0ut5wc5nmu0tacjc9pyyahdyq5ujghwljkdxr...,['stake1uyx0ut5wc5nmu0tacjc9pyyahdyq5ujghwljkd...,['37f5682e2622c821175bab9d5637b7321ecacc5ec663...,[],relay.zenithpool.io,31400,https://metadata.zenithpool.io,21cc44498ab6e74e081a805e629fff171a66d0c8d034f9...,ZEN,ZENITH,"One owner, one operator, one pool. Operating s...",https://zenithpool.io,193.122.12.254,-122.333519,47.611012,98101,Washington,WA,"{'asn': 31898, 'isp': 'Oracle Corporation'}",Seattle,United States,US,NaN,NaN,NaN,Oracle Corporation,31898,5.672152e+05,546041.471277,50100.0,11.321660
3,pool1q80jjs53w0fx836n8g38gtdwr8ck5zre3da90peux...,01df29429173d263c7533a22742dae19f16a08798b7a57...,b75115a1e4e32a33985fc720f3715f0e99617b448fbc36...,259,1735454735054,7.345360e-05,2.590266e-02,724,1735713797295,7.315390e-05,200000000000,207011830413,0.005,340000000,stake1uxxsa2rlkv728pve3ks7kn7c6v68luagdjgwa0me...,['stake1uxxsa2rlkv728pve3ks7kn7c6v68luagdjgwa0...,['e8951eddb19b697bc74549aa9d4f956a8be8d5cf2c7a...,[],95.183.53.20,6060,https://ispool.live/metadata,34fdde237812fab14d29a80423bb295f39122f4fea1aae...,000,Switzerland Investment,"Stability, Security, Reliability, Neutrality! ...",https://ispool.live/,95.183.53.20,8.536950,47.374168,8043,Zurich,ZH,"{'asn': 197988, 'isp': 'Solar Communications G...",Zürich,Switzerland,CH,NaN,NaN,NaN,Solar Communications Gmbh,197988,1.735455e+06,207011.830413,200000.0,8.677274
5,pool1ddskftmsscw92d7vnj89pldwx5feegkgcmamgt5t0...,6b6164af70861c5537cc9c8e50fdae35139ca2c8c6fbb4...,78b0b08fec7458afd1209dcd26bd0c10faa640a49d5013...,23,4000924534,1.693000e-07,5.971610e-05,14,4000924534,1.686000e-07,7149000000,0,0.050,340000000,stake1u9w20uzsevkxe6vp2

In [56]:

df_ada_staked_per_pool['live_saturation_%_value'] = df_ada_staked_per_pool.live_saturation * 100
df_ada_staked_per_pool

,pool_id,hex,vrf_key,blocks_minted,live_stake,live_size,live_saturation,live_delegators,active_stake,active_size,declared_pledge,live_pledge,margin_cost,fixed_cost,reward_account,owners,registration,retirement,ip_host,port,url,hash,ticker,name,description,homepage,resolved_ip,longitude,latitude,zip,region_name,region_code,connection,city,country_name,country_code,error,success,detail,isp,asn,live_stake_ada,live_pledge_ada,live_declared_pledge_ada,pledge_ratio,live_saturation_value,live_saturation_%_value
0,pool1z5uqdk7dzdxaae5633fqfcu2eqzy3a3rgtuvy087f...,153806dbcd134ddee69a8c5204e38ac80448f62342f8c2...,0220a5d08adbfe9554b52d7b2993be5892ac3ff340e674...,4097,32128532235090,1.359849e-03,4.795368e-01,3170,32462689016256,1.368182e-03,100000000000,574564954842,0.009,340000000,stake1uy89kzrdlpaz5rzu8x95r4qnlpqhd3f8mf09edjp...,['stake1uy89kzrdlpaz5rzu8x95r4qnlpqhd3f8mf09ed...,['841cca81da918feb9fa7257a34630eac95794be712ed...,[],54.220.20.40,3002,https://raw.githubusercontent.com/Octalus/card...,ca7d12decf886e31f5226b5946c62edc81a7e40af95ce7...,OCTAS,OctasPool,Octa's Performance Pool,https://octaluso.dyndns.org,54.220.20.40,-6.243330,53.353889,None,Leinster,L,"{'asn': 16509, 'isp': 'amazon.com, Inc'}",Dublin,Ireland,IE,NaN,NaN,NaN,"amazon.com, Inc",16509,3.212853e+07,574564.954842,100000.0,321.285322,47.953685,47.953685
1,pool1pu5jlj4q9w9jlxeu370a3c9myx47md5j5m2str0na...,0f292fcaa02b8b2f9b3c8f9fd8e0bb21abedb692a6d505...,b512cc7c1a8ba689c2d8fd27adfdbac2049a3f8f95c8b8...,1824,19427988062303,8.222951e-04,2.899739e-01,170,19414895397699,8.182658e-04,510000000000,567027531192,0.049,340000000,stake1u98nnlkvkk23vtvf9273uq7cph5ww6u2yq2389ps...,['stake1u98nnlkvkk23vtvf9273uq7cph5ww6u2yq2389...,['a96c79773b7506211eb56bf94886a2face17657d1009...,[],relays.mainnet.stakenuts.com,3001,https://stakenuts.com/mainnet.json,47c0c68cb57f4a5b4a87bad896fc274678e7aea98e200f...,NUTS,StakeNuts,StakeNuts.com,https://stakenuts.com/,198.13.62.211,139.746155,35.583511,143-0006,Kanagawa,14,"{'asn': 20473, 'isp': 'The Constant Company LLC'}",Kawasaki,Japan,JP,NaN,NaN,NaN,The Constant Company LLC,20473,1.942799e+07,567027.531192,510000.0,38.094094,28.997391,28.997391
2,pool1c8k78ny3xvsfgenhf4yzvpzwgzxmz0t0um0h2xnn2...,c1ede3cc9133209466774d4826044e408db13d6fe6df75...,b8ba5f7dcbf17e95781e325114da46def1fa908435a27d...,80,567215180144,2.400750e-05,8.466013e-03,17,555030759623,2.339250e-05,50100000000,546041471277,0.000,340000000,stake1uyx0ut5wc5nmu0tacjc9pyyahdyq5ujghwljkdxr...,['stake1uyx0ut5wc5nmu0tacjc9pyyahdyq5ujghwljkd...,['37f5682e2622c821175bab9d5637b7321ecacc5ec663...,[],relay.zenithpool.io,31400,https://metadata.zenithpool.io,21cc44498ab6e74e081a805e629fff171a66d0c8d034f9...,ZEN,ZENITH,"One owner, one operator, one pool. Operating s...",https://zenithpool.io,193.122.12.254,-122.333519,47.611012,98101,Washington,WA,"{'asn': 31898, 'isp': 'Oracle Corporation'}",Seattle,United States,US,NaN,NaN,NaN,Oracle Corporation,31898,5.672152e+05,546041.471277,50100.0,11.321660,0.846601,0.846601
3,pool1q80jjs53w0fx836n8g38gtdwr8ck5zre3da90peux...,01df29429173d263c7533a22742dae19f16a08798b7a57...,b75115a1e4e32a33985fc720f3715f0e99617b448fbc36...,259,1735454735054,7.345360e-05,2.590266e-02,724,1735713797295,7.315390e-05,200000000000,207011830413,0.005,340000000,stake1uxxsa2rlkv728pve3ks7kn7c6v68luagdjgwa0me...,['stake1uxxsa2rlkv728pve3ks7kn7c6v68luagdjgwa0...,['e8951eddb19b697bc74549aa9d4f956a8be8d5cf2c7a...,[],95.183.53.20,6060,https://ispool.live/metadata,34fdde237812fab14d29a80423bb295f39122f4fea1aae...,000,Switzerland Investment,"Stability, Security, Reliability, Neutrality! ...",https://ispool.live/,95.183.53.20,8.536950,47.374168,8043,Zurich,ZH,"{'asn': 197988, 'isp': 'Solar Communications G...",Zürich,Switzerland,CH,NaN,NaN,NaN,Solar Communications Gmbh,197988,1.735455e+06,207011.830413,200000.0,8.677274,2.590266,2.590266
5,pool1ddskftmsscw92d7vnj89pldwx5feegkgcmamgt5t0...,6b6164af70861c5537cc9c8e50fdae35139ca2c8c6fbb4...,78b0b08fec7458afd1209dcd26bd0c10faa640a49d5013.

In [60]:
df_ada_staked_per_pool.drop('live_saturation_value', axis=1, inplace=True)

In [62]:
df_ada_staked_per_pool.drop(['error','success','detail'] , axis=1, inplace=True)

In [63]:
df_ada_staked_per_p

,pool_id,hex,vrf_key,blocks_minted,live_stake,live_size,live_saturation,live_delegators,active_stake,active_size,declared_pledge,live_pledge,margin_cost,fixed_cost,reward_account,owners,registration,retirement,ip_host,port,url,hash,ticker,name,description,homepage,resolved_ip,longitude,latitude,zip,region_name,region_code,connection,city,country_name,country_code,isp,asn,live_stake_ada,live_pledge_ada,live_declared_pledge_ada,pledge_ratio,live_saturation_%_value
0,pool1z5uqdk7dzdxaae5633fqfcu2eqzy3a3rgtuvy087f...,153806dbcd134ddee69a8c5204e38ac80448f62342f8c2...,0220a5d08adbfe9554b52d7b2993be5892ac3ff340e674...,4097,32128532235090,1.359849e-03,4.795368e-01,3170,32462689016256,1.368182e-03,100000000000,574564954842,0.009,340000000,stake1uy89kzrdlpaz5rzu8x95r4qnlpqhd3f8mf09edjp...,['stake1uy89kzrdlpaz5rzu8x95r4qnlpqhd3f8mf09ed...,['841cca81da918feb9fa7257a34630eac95794be712ed...,[],54.220.20.40,3002,https://raw.githubusercontent.com/Octalus/card...,ca7d12decf886e31f5226b5946c62edc81a7e40af95ce7...,OCTAS,OctasPool,Octa's Performance Pool,https://octaluso.dyndns.org,54.220.20.40,-6.243330,53.353889,None,Leinster,L,"{'asn': 16509, 'isp': 'amazon.com, Inc'}",Dublin,Ireland,IE,"amazon.com, Inc",16509,3.212853e+07,574564.954842,100000.0,321.285322,47.953685
1,pool1pu5jlj4q9w9jlxeu370a3c9myx47md5j5m2str0na...,0f292fcaa02b8b2f9b3c8f9fd8e0bb21abedb692a6d505...,b512cc7c1a8ba689c2d8fd27adfdbac2049a3f8f95c8b8...,1824,19427988062303,8.222951e-04,2.899739e-01,170,19414895397699,8.182658e-04,510000000000,567027531192,0.049,340000000,stake1u98nnlkvkk23vtvf9273uq7cph5ww6u2yq2389ps...,['stake1u98nnlkvkk23vtvf9273uq7cph5ww6u2yq2389...,['a96c79773b7506211eb56bf94886a2face17657d1009...,[],relays.mainnet.stakenuts.com,3001,https://stakenuts.com/mainnet.json,47c0c68cb57f4a5b4a87bad896fc274678e7aea98e200f...,NUTS,StakeNuts,StakeNuts.com,https://stakenuts.com/,198.13.62.211,139.746155,35.583511,143-0006,Kanagawa,14,"{'asn': 20473, 'isp': 'The Constant Company LLC'}",Kawasaki,Japan,JP,The Constant Company LLC,20473,1.942799e+07,567027.531192,510000.0,38.094094,28.997391
2,pool1c8k78ny3xvsfgenhf4yzvpzwgzxmz0t0um0h2xnn2...,c1ede3cc9133209466774d4826044e408db13d6fe6df75...,b8ba5f7dcbf17e95781e325114da46def1fa908435a27d...,80,567215180144,2.400750e-05,8.466013e-03,17,555030759623,2.339250e-05,50100000000,546041471277,0.000,340000000,stake1uyx0ut5wc5nmu0tacjc9pyyahdyq5ujghwljkdxr...,['stake1uyx0ut5wc5nmu0tacjc9pyyahdyq5ujghwljkd...,['37f5682e2622c821175bab9d5637b7321ecacc5ec663...,[],relay.zenithpool.io,31400,https://metadata.zenithpool.io,21cc44498ab6e74e081a805e629fff171a66d0c8d034f9...,ZEN,ZENITH,"One owner, one operator, one pool. Operating s...",https://zenithpool.io,193.122.12.254,-122.333519,47.611012,98101,Washington,WA,"{'asn': 31898, 'isp': 'Oracle Corporation'}",Seattle,United States,US,Oracle Corporation,31898,5.672152e+05,546041.471277,50100.0,11.321660,0.846601
3,pool1q80jjs53w0fx836n8g38gtdwr8ck5zre3da90peux...,01df29429173d263c7533a22742dae19f16a08798b7a57...,b75115a1e4e32a33985fc720f3715f0e99617b448fbc36...,259,1735454735054,7.345360e-05,2.590266e-02,724,1735713797295,7.315390e-05,200000000000,207011830413,0.005,340000000,stake1uxxsa2rlkv728pve3ks7kn7c6v68luagdjgwa0me...,['stake1uxxsa2rlkv728pve3ks7kn7c6v68luagdjgwa0...,['e8951eddb19b697bc74549aa9d4f956a8be8d5cf2c7a...,[],95.183.53.20,6060,https://ispool.live/metadata,34fdde237812fab14d29a80423bb295f39122f4fea1aae...,000,Switzerland Investment,"Stability, Security, Reliability, Neutrality! ...",https://ispool.live/,95.183.53.20,8.536950,47.374168,8043,Zurich,ZH,"{'asn': 197988, 'isp': 'Solar Communications G...",Zürich,Switzerland,CH,Solar Communications Gmbh,197988,1.735455e+06,207011.830413,200000.0,8.677274,2.590266
5,pool1ddskftmsscw92d7vnj89pldwx5feegkgcmamgt5t0...,6b6164af70861c5537cc9c8e50fdae35139ca2c8c6fbb4...,78b0b08fec7458afd1209dcd26bd0c10faa640a49d5013...,23,4000924534,1.693000e-07,5.971610e-05,14,4000924534,1.686000e-07,7149000000,0,0.050,340000000,stake1u9w20uzsevkxe6vp2nlmqjjr

In [65]:
df_copy = df_ada_staked_per_pool.copy()
# need to map our dfy multipool_ration to the df_copy based on homepage
df_copy['multipool_ratio'] = df_copy['homepage'].map(dfy['multipool_ratio'])
df_copy

,pool_id,hex,vrf_key,blocks_minted,live_stake,live_size,live_saturation,live_delegators,active_stake,active_size,declared_pledge,live_pledge,margin_cost,fixed_cost,reward_account,owners,registration,retirement,ip_host,port,url,hash,ticker,name,description,homepage,resolved_ip,longitude,latitude,zip,region_name,region_code,connection,city,country_name,country_code,isp,asn,live_stake_ada,live_pledge_ada,live_declared_pledge_ada,pledge_ratio,live_saturation_%_value,multipool_ratio
0,pool1z5uqdk7dzdxaae5633fqfcu2eqzy3a3rgtuvy087f...,153806dbcd134ddee69a8c5204e38ac80448f62342f8c2...,0220a5d08adbfe9554b52d7b2993be5892ac3ff340e674...,4097,32128532235090,1.359849e-03,4.795368e-01,3170,32462689016256,1.368182e-03,100000000000,574564954842,0.009,340000000,stake1uy89kzrdlpaz5rzu8x95r4qnlpqhd3f8mf09edjp...,['stake1uy89kzrdlpaz5rzu8x95r4qnlpqhd3f8mf09ed...,['841cca81da918feb9fa7257a34630eac95794be712ed...,[],54.220.20.40,3002,https://raw.githubusercontent.com/Octalus/card...,ca7d12decf886e31f5226b5946c62edc81a7e40af95ce7...,OCTAS,OctasPool,Octa's Performance Pool,https://octaluso.dyndns.org,54.220.20.40,-6.243330,53.353889,None,Leinster,L,"{'asn': 16509, 'isp': 'amazon.com, Inc'}",Dublin,Ireland,IE,"amazon.com, Inc",16509,3.212853e+07,574564.954842,100000.0,321.285322,47.953685,0.039494
1,pool1pu5jlj4q9w9jlxeu370a3c9myx47md5j5m2str0na...,0f292fcaa02b8b2f9b3c8f9fd8e0bb21abedb692a6d505...,b512cc7c1a8ba689c2d8fd27adfdbac2049a3f8f95c8b8...,1824,19427988062303,8.222951e-04,2.899739e-01,170,19414895397699,8.182658e-04,510000000000,567027531192,0.049,340000000,stake1u98nnlkvkk23vtvf9273uq7cph5ww6u2yq2389ps...,['stake1u98nnlkvkk23vtvf9273uq7cph5ww6u2yq2389...,['a96c79773b7506211eb56bf94886a2face17657d1009...,[],relays.mainnet.stakenuts.com,3001,https://stakenuts.com/mainnet.json,47c0c68cb57f4a5b4a87bad896fc274678e7aea98e200f...,NUTS,StakeNuts,StakeNuts.com,https://stakenuts.com/,198.13.62.211,139.746155,35.583511,143-0006,Kanagawa,14,"{'asn': 20473, 'isp': 'The Constant Company LLC'}",Kawasaki,Japan,JP,The Constant Company LLC,20473,1.942799e+07,567027.531192,510000.0,38.094094,28.997391,0.039494
2,pool1c8k78ny3xvsfgenhf4yzvpzwgzxmz0t0um0h2xnn2...,c1ede3cc9133209466774d4826044e408db13d6fe6df75...,b8ba5f7dcbf17e95781e325114da46def1fa908435a27d...,80,567215180144,2.400750e-05,8.466013e-03,17,555030759623,2.339250e-05,50100000000,546041471277,0.000,340000000,stake1uyx0ut5wc5nmu0tacjc9pyyahdyq5ujghwljkdxr...,['stake1uyx0ut5wc5nmu0tacjc9pyyahdyq5ujghwljkd...,['37f5682e2622c821175bab9d5637b7321ecacc5ec663...,[],relay.zenithpool.io,31400,https://metadata.zenithpool.io,21cc44498ab6e74e081a805e629fff171a66d0c8d034f9...,ZEN,ZENITH,"One owner, one operator, one pool. Operating s...",https://zenithpool.io,193.122.12.254,-122.333519,47.611012,98101,Washington,WA,"{'asn': 31898, 'isp': 'Oracle Corporation'}",Seattle,United States,US,Oracle Corporation,31898,5.672152e+05,546041.471277,50100.0,11.321660,0.846601,0.039494
3,pool1q80jjs53w0fx836n8g38gtdwr8ck5zre3da90peux...,01df29429173d263c7533a22742dae19f16a08798b7a57...,b75115a1e4e32a33985fc720f3715f0e99617b448fbc36...,259,1735454735054,7.345360e-05,2.590266e-02,724,1735713797295,7.315390e-05,200000000000,207011830413,0.005,340000000,stake1uxxsa2rlkv728pve3ks7kn7c6v68luagdjgwa0me...,['stake1uxxsa2rlkv728pve3ks7kn7c6v68luagdjgwa0...,['e8951eddb19b697bc74549aa9d4f956a8be8d5cf2c7a...,[],95.183.53.20,6060,https://ispool.live/metadata,34fdde237812fab14d29a80423bb295f39122f4fea1aae...,000,Switzerland Investment,"Stability, Security, Reliability, Neutrality! ...",https://ispool.live/,95.183.53.20,8.536950,47.374168,8043,Zurich,ZH,"{'asn': 197988, 'isp': 'Solar Communications G...",Zürich,Switzerland,CH,Solar Communications Gmbh,197988,1.735455e+06,207011.830413,200000.0,8.677274,2.590266,0.039494
5,pool1ddskftmsscw92d7vnj89pldwx5feegkgcmamgt5t0...,6b6164af70861c5537cc9c8e50fdae35139ca2c8c6fbb4...,78b0b08fec7458afd1209dcd26bd0c10faa640a49d5013...,23,4000924534,1.693000e-07,5.971610e-05,14,4000924534,1.686000e-07,7149000

In [66]:
df_copy.multipool_ratio.fillna(0, inplace=True)

In [67]:
df_copy

,pool_id,hex,vrf_key,blocks_minted,live_stake,live_size,live_saturation,live_delegators,active_stake,active_size,declared_pledge,live_pledge,margin_cost,fixed_cost,reward_account,owners,registration,retirement,ip_host,port,url,hash,ticker,name,description,homepage,resolved_ip,longitude,latitude,zip,region_name,region_code,connection,city,country_name,country_code,isp,asn,live_stake_ada,live_pledge_ada,live_declared_pledge_ada,pledge_ratio,live_saturation_%_value,multipool_ratio
0,pool1z5uqdk7dzdxaae5633fqfcu2eqzy3a3rgtuvy087f...,153806dbcd134ddee69a8c5204e38ac80448f62342f8c2...,0220a5d08adbfe9554b52d7b2993be5892ac3ff340e674...,4097,32128532235090,1.359849e-03,4.795368e-01,3170,32462689016256,1.368182e-03,100000000000,574564954842,0.009,340000000,stake1uy89kzrdlpaz5rzu8x95r4qnlpqhd3f8mf09edjp...,['stake1uy89kzrdlpaz5rzu8x95r4qnlpqhd3f8mf09ed...,['841cca81da918feb9fa7257a34630eac95794be712ed...,[],54.220.20.40,3002,https://raw.githubusercontent.com/Octalus/card...,ca7d12decf886e31f5226b5946c62edc81a7e40af95ce7...,OCTAS,OctasPool,Octa's Performance Pool,https://octaluso.dyndns.org,54.220.20.40,-6.243330,53.353889,None,Leinster,L,"{'asn': 16509, 'isp': 'amazon.com, Inc'}",Dublin,Ireland,IE,"amazon.com, Inc",16509,3.212853e+07,574564.954842,100000.0,321.285322,47.953685,0.039494
1,pool1pu5jlj4q9w9jlxeu370a3c9myx47md5j5m2str0na...,0f292fcaa02b8b2f9b3c8f9fd8e0bb21abedb692a6d505...,b512cc7c1a8ba689c2d8fd27adfdbac2049a3f8f95c8b8...,1824,19427988062303,8.222951e-04,2.899739e-01,170,19414895397699,8.182658e-04,510000000000,567027531192,0.049,340000000,stake1u98nnlkvkk23vtvf9273uq7cph5ww6u2yq2389ps...,['stake1u98nnlkvkk23vtvf9273uq7cph5ww6u2yq2389...,['a96c79773b7506211eb56bf94886a2face17657d1009...,[],relays.mainnet.stakenuts.com,3001,https://stakenuts.com/mainnet.json,47c0c68cb57f4a5b4a87bad896fc274678e7aea98e200f...,NUTS,StakeNuts,StakeNuts.com,https://stakenuts.com/,198.13.62.211,139.746155,35.583511,143-0006,Kanagawa,14,"{'asn': 20473, 'isp': 'The Constant Company LLC'}",Kawasaki,Japan,JP,The Constant Company LLC,20473,1.942799e+07,567027.531192,510000.0,38.094094,28.997391,0.039494
2,pool1c8k78ny3xvsfgenhf4yzvpzwgzxmz0t0um0h2xnn2...,c1ede3cc9133209466774d4826044e408db13d6fe6df75...,b8ba5f7dcbf17e95781e325114da46def1fa908435a27d...,80,567215180144,2.400750e-05,8.466013e-03,17,555030759623,2.339250e-05,50100000000,546041471277,0.000,340000000,stake1uyx0ut5wc5nmu0tacjc9pyyahdyq5ujghwljkdxr...,['stake1uyx0ut5wc5nmu0tacjc9pyyahdyq5ujghwljkd...,['37f5682e2622c821175bab9d5637b7321ecacc5ec663...,[],relay.zenithpool.io,31400,https://metadata.zenithpool.io,21cc44498ab6e74e081a805e629fff171a66d0c8d034f9...,ZEN,ZENITH,"One owner, one operator, one pool. Operating s...",https://zenithpool.io,193.122.12.254,-122.333519,47.611012,98101,Washington,WA,"{'asn': 31898, 'isp': 'Oracle Corporation'}",Seattle,United States,US,Oracle Corporation,31898,5.672152e+05,546041.471277,50100.0,11.321660,0.846601,0.039494
3,pool1q80jjs53w0fx836n8g38gtdwr8ck5zre3da90peux...,01df29429173d263c7533a22742dae19f16a08798b7a57...,b75115a1e4e32a33985fc720f3715f0e99617b448fbc36...,259,1735454735054,7.345360e-05,2.590266e-02,724,1735713797295,7.315390e-05,200000000000,207011830413,0.005,340000000,stake1uxxsa2rlkv728pve3ks7kn7c6v68luagdjgwa0me...,['stake1uxxsa2rlkv728pve3ks7kn7c6v68luagdjgwa0...,['e8951eddb19b697bc74549aa9d4f956a8be8d5cf2c7a...,[],95.183.53.20,6060,https://ispool.live/metadata,34fdde237812fab14d29a80423bb295f39122f4fea1aae...,000,Switzerland Investment,"Stability, Security, Reliability, Neutrality! ...",https://ispool.live/,95.183.53.20,8.536950,47.374168,8043,Zurich,ZH,"{'asn': 197988, 'isp': 'Solar Communications G...",Zürich,Switzerland,CH,Solar Communications Gmbh,197988,1.735455e+06,207011.830413,200000.0,8.677274,2.590266,0.039494
5,pool1ddskftmsscw92d7vnj89pldwx5feegkgcmamgt5t0...,6b6164af70861c5537cc9c8e50fdae35139ca2c8c6fbb4...,78b0b08fec7458afd1209dcd26bd0c10faa640a49d5013...,23,4000924534,1.693000e-07,5.971610e-05,14,4000924534,1.686000e-07,7149000